# Fine-tuning BETO on CodiEsp-D

In this notebook, following a multi-label sequence classification approach, the BETO model is fine-tuned on both the training and development sets of the CodiEsp-D corpus. Additionally, the predictions made by the model on the test set are saved, in order to futher evaluate the clinical coding performance of the model (see `results/CodiEsp-D/Evaluation.ipynb`).

In [1]:
import tensorflow as tf

# Auxiliary components
import sys
sys.path.append("..")
from nlp_utils import *

# BETO tokenizer
from keras_bert import load_vocabulary, Tokenizer
model_path = "BETO_cased/"
config_path = model_path + "config.json"
checkpoint_path = model_path + "model.ckpt-2000000"
vocab_file = "vocab.txt"
tokenizer = Tokenizer(token_dict=load_vocabulary(model_path + vocab_file), pad_index=1, cased=True)

# Hyper-parameters
text_col = "raw_text"
training = False
trainable = True
SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS = 24
LR = 3e-5
train_weight = 4
all_abs_weight = 1

random_seed = 0
tf.set_random_seed(random_seed)

Using TensorFlow backend.


## Load text

Firstly, all text files from training and development CodiEsp corpora are loaded in different dataframes.

Also, CIE-Diagnóstico codes are loaded.

In [2]:
corpus_path = "../datasets/codiesp_v4/"
abs_corpus_path = "../datasets/abstractsWithCIE10_v2/"

### Training corpus

In [3]:
%%time
train_path = corpus_path + "train/text_files/"
train_files = [f for f in os.listdir(train_path) if os.path.isfile(train_path + f)]
train_data = load_text_files(train_files, train_path)
df_text_train = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in train_files], 'raw_text': train_data})

CPU times: user 0 ns, sys: 10.6 ms, total: 10.6 ms
Wall time: 10.3 ms


In [4]:
df_text_train.shape

(500, 2)

In [5]:
df_text_train.head()

,doc_id,raw_text
0,S0365-66912006000600011-1,Un varón de 13 años es remitido para valoració...
1,S1139-13752009000200010-2,Paciente de 42 años diagnosticado de pancoliti...
2,S1130-05582017000100037-1,"Varón de 72 años, sin antecedentes médicos de ..."
3,S1139-76322016000300015-1,Lactante de ocho días cuyos padres consultan p...
4,S0211-69952011000100019-1,Mujer de 47 años de edad con antecedentes de g...


In [6]:
df_text_train.raw_text[0]

'Un varón de 13 años es remitido para valoración oftalmológica por mala visión. Fenotípicamente era un niño de talla corta con una estatura de 133 cm, braquimorfia y braquidactilia en las cuatro extremidades.\nEl paciente presentaba un error refractivo corregido de -13,00 -6,50 a 1º en el ojo derecho y de -16,00-6,25 a 179º en el izquierdo. Con dicha corrección alcanzaba una agudeza visual de 0,4 y 0,2 respectivamente. No existía diplopía monocular ni hallazgos en la motilidad ocular extrínseca e intrínseca.\nEl diámetro corneal horizontal era de 12,0 mm en ambos ojos y la paquimetría de 613 y 611 micras respectivamente. La cámara anterior era estrecha, apreciándose iridofacodonesis bilateral. Se evidenció microesferofaquia con desplazamiento anterior de ambos cristalinos dentro de la cámara posterior.\nLa presión intraocular era de 20 mmHg bilateralmente. Gonioscópicamente se apreció un ángulo estrecho simétrico en ambos ojos grado II según Schaffer.\nLa exploración mediante topógrafo

We also load the CIE-Diagnóstico codes table:

In [7]:
df_codes_train = pd.read_table(corpus_path + "train/trainD.tsv", sep='\t', header=None)

In [8]:
df_codes_train.columns = ["doc_id", "code"]

In [9]:
df_codes_train.shape

(5639, 2)

In [10]:
df_codes_train.head()

,doc_id,code
0,S0004-06142005000700014-1,n44.8
1,S0004-06142005000700014-1,z20.818
2,S0004-06142005000700014-1,r60.9
3,S0004-06142005000700014-1,r52
4,S0004-06142005000700014-1,a23.9


In [11]:
len(set(df_codes_train["doc_id"]))

500

### Development corpus

In [12]:
%%time
dev_path = corpus_path + "dev/text_files/"
dev_files = [f for f in os.listdir(dev_path) if os.path.isfile(dev_path + f)]
dev_data = load_text_files(dev_files, dev_path)
df_text_dev = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in dev_files], 'raw_text': dev_data})

CPU times: user 6.24 ms, sys: 0 ns, total: 6.24 ms
Wall time: 5.85 ms


In [13]:
df_text_dev.shape

(250, 2)

In [14]:
df_text_dev.head()

,doc_id,raw_text
0,S1130-63432016000600013-1,"Varón de 75 años, con antecedentes de hiperuri..."
1,S0365-66912003000600010-1,"Paciente de 33 años de edad, gestante de 34 se..."
2,S0211-69952012000200030-1,Mujer de 67 años con múltiples factores de rie...
3,S0365-66912004000900009-1,Paciente de 55 años que acudió a urgencias por...
4,S1139-76322016000300016-2,"Lactante de 1 mes y 29 días, sin antecedentes ..."


In [15]:
df_text_dev.raw_text[0]

'Varón de 75 años, con antecedentes de hiperuricemia en tratamiento con Alopurinol que ingresa para realización de resección transuretral de próstata.\nPostoperatorio inmediato sin incidencias con tratamiento con Pantoprazol, Ciprofloxacino, Paracetamol, Enantyum y Alopurinol. Al cuarto día de postoperatorio presenta mareos, temblor con componente mioclónico en extremidades y tronco e incapacidad para caminar, sin verse alteraciones analíticas. En esta situación se pauta Rivotril y se suspende el tratamiento con Ciprofloxacino, desapareciendo la clínica mioclónica y mejorando el estado del paciente, por lo que se decide el alta hospitalaria.\n\n'

We also load the CIE-Diagnóstico codes table:

In [16]:
df_codes_dev = pd.read_table(corpus_path + "dev/devD.tsv", sep='\t', header=None)

In [17]:
df_codes_dev.columns = ["doc_id", "code"]

In [18]:
df_codes_dev.shape

(2677, 2)

In [19]:
df_codes_dev.head()

,doc_id,code
0,S0004-06142005000900016-1,q62.11
1,S0004-06142005000900016-1,n28.89
2,S0004-06142005000900016-1,n39.0
3,S0004-06142005000900016-1,r31.9
4,S0004-06142005000900016-1,n23


In [20]:
len(set(df_codes_dev["doc_id"]))

250

### Train-Dev abstracts corpus

From the additional abstracts corpus, we load the text from the abstracts containing CIE-Diagnóstico codes which are present either in the training or development CodiEsp-D corpora:

In [21]:
%%time
df_text_all_abs = pd.read_table(abs_corpus_path + "train_dev_abstracts_text.tsv", sep='\t')

CPU times: user 1.32 s, sys: 160 ms, total: 1.48 s
Wall time: 1.48 s


In [22]:
df_text_all_abs.shape

(100397, 3)

We only select the abstracts with a subtokens sequence length <= the maximum input sequence size used by the BETO model (128 subtokens):

In [23]:
all_abs_doc_one_frag = set(pd.read_table(abs_corpus_path + "all_abstracts_seq_len_beto_128.tsv", 
                                               sep='\t', header=None)[0])

In [24]:
len(all_abs_doc_one_frag)

23377

In [25]:
df_text_all_abs = df_text_all_abs[df_text_all_abs["doc_id"].isin(all_abs_doc_one_frag)]

In [26]:
df_text_all_abs.shape

(15258, 3)

In [27]:
df_text_all_abs.head()

,doc_id,raw_text,punc_text
24,biblio-1000299,INTRODUCCIÓN: Las complicaciones del tratamien...,INTRODUCCIÓN Las complicaciones del tratamient...
38,biblio-1000756,Este libro es el resultado de un trabajo minuc...,Este libro es el resultado de un trabajo minuc...
48,biblio-1002637,Se efectuó una revisión actualizada sobre el d...,Se efectuó una revisión actualizada sobre el d...
94,biblio-1004283,RESUMEN El insomnio es el trastorno del sueño ...,RESUMEN El insomnio es el trastorno del sueño ...
105,biblio-1005037,La microlitiasis testicular (TM) es una patolo...,La microlitiasis testicular TM es una patologí...


We also load the CIE-Diagnóstico codes from the previously loaded abstracts:

In [28]:
df_codes_d_all_abs = pd.read_table(abs_corpus_path + "train_dev_abstracts_codes.tsv", sep='\t', 
                                   header=None)

In [29]:
df_codes_d_all_abs.columns = ["doc_id", "code"]

In [30]:
df_codes_d_all_abs = df_codes_d_all_abs[df_codes_d_all_abs["doc_id"].isin(all_abs_doc_one_frag)]

In [31]:
df_codes_d_all_abs.shape

(21016, 2)

In [32]:
df_codes_d_all_abs.head()

,doc_id,code
1,lil-286177,i82.40
2,lil-286177,i82.90
9,lil-273480,g31.9
10,lil-506160,q03.1
45,lil-176866,g51.0


We join the training and development CodiEsp as well as the abstracts codes dataframes together:

In [33]:
df_codes_train_dev_abs = pd.concat([df_codes_train, df_codes_dev, df_codes_d_all_abs])

In [34]:
df_codes_train_dev_abs.shape

(29332, 2)

In [35]:
df_codes_train_dev_abs.head()

,doc_id,code
0,S0004-06142005000700014-1,n44.8
1,S0004-06142005000700014-1,z20.818
2,S0004-06142005000700014-1,r60.9
3,S0004-06142005000700014-1,r52
4,S0004-06142005000700014-1,a23.9


## Creating corpora of annotated sentences

Leveraging the information available for the named-entity-recognition and normalization (NER-N) CodiEsp-X task, we create both a training and a development corpus of annotated sentences with CIE-Diagnóstico codes.

Firstly, we pre-process the NER-N precedure-codes annotations available for both the training and development corpora.

In [36]:
# Training corpus

In [37]:
%%time

codiesp_x_train = pd.read_table(corpus_path + "train/trainX.tsv", sep='\t', header=None)

CPU times: user 7.7 ms, sys: 3.53 ms, total: 11.2 ms
Wall time: 10.9 ms


In [38]:
codiesp_x_train.columns = ["doc_id", "type", "code", "word", "location"]

In [39]:
codiesp_x_train.shape

(9181, 5)

In [40]:
codiesp_x_train.head()

,doc_id,type,code,word,location
0,S0004-06142005000700014-1,PROCEDIMIENTO,bw03zzz,Rx tórax,2163 2171
1,S0004-06142005000700014-1,PROCEDIMIENTO,3e02329,Estreptomicina intramuscular,2787 2801;2810 2823
2,S0004-06142005000700014-1,DIAGNOSTICO,n44.8,teste derecho aumentado de tamaño,1343 1376
3,S0004-06142005000700014-1,DIAGNOSTICO,z20.818,exposición a Brucella,594 615
4,S0004-06142005000700014-1,DIAGNOSTICO,r60.9,edemas,1250 1256


In [41]:
codiesp_x_train = codiesp_x_train[codiesp_x_train["type"] == "DIAGNOSTICO"]

In [42]:
codiesp_x_train.shape

(7209, 5)

In [43]:
df_codes_train_ner = process_ner_labels(codiesp_x_train).sort_values(["doc_id", "start", "end"])

In [44]:
df_codes_train_ner.head()

,doc_id,type,code,word,start,end
3,S0004-06142005000700014-1,DIAGNOSTICO,r52,dolores,78,85
13,S0004-06142005000700014-1,DIAGNOSTICO,m25.50,dolores osteoarticulares,78,102
9,S0004-06142005000700014-1,DIAGNOSTICO,r50.9,fiebre,147,153
14,S0004-06142005000700014-1,DIAGNOSTICO,a23.9,brucella,360,368
10,S0004-06142005000700014-1,DIAGNOSTICO,r50.9,síndrome febril,534,549


In [45]:
df_codes_train_ner.shape

(8272, 6)

In [46]:
# Development corpus

In [47]:
%%time

codiesp_x_dev = pd.read_table(corpus_path + "dev/devX.tsv", sep='\t', header=None)

CPU times: user 7.57 ms, sys: 0 ns, total: 7.57 ms
Wall time: 7.47 ms


In [48]:
codiesp_x_dev.columns = ["doc_id", "type", "code", "word", "location"]

In [49]:
codiesp_x_dev.shape

(4477, 5)

In [50]:
codiesp_x_dev.head()

,doc_id,type,code,word,location
0,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,307 316;348 361
1,S0004-06142005000900016-1,PROCEDIMIENTO,ct13,gammagrafía renal,739 756
2,S0004-06142005000900016-1,DIAGNOSTICO,q62.11,estenosis en la unión pieloureteral derecha,540 583
3,S0004-06142005000900016-1,DIAGNOSTICO,n28.89,ectasia pielocalicial,326 347
4,S0004-06142005000900016-1,DIAGNOSTICO,n39.0,infecciones del tracto urinario,198 229


In [51]:
codiesp_x_dev = codiesp_x_dev[codiesp_x_dev["type"] == "DIAGNOSTICO"]

In [52]:
codiesp_x_dev.shape

(3431, 5)

In [53]:
df_codes_dev_ner = process_ner_labels(codiesp_x_dev).sort_values(["doc_id", "start", "end"])

In [54]:
df_codes_dev_ner.head()

,doc_id,type,code,word,start,end
11,S0004-06142005000900016-1,DIAGNOSTICO,k26.9,ulcus duodenal,37,51
14,S0004-06142005000900016-1,DIAGNOSTICO,k59.00,estreñimiento,54,67
4,S0004-06142005000900016-1,DIAGNOSTICO,n23,dolor en fosa renal,85,104
5,S0004-06142005000900016-1,DIAGNOSTICO,n28.0,crisis renoureteral,128,147
13,S0004-06142005000900016-1,DIAGNOSTICO,n20.0,nefrolitiasis,168,181


In [55]:
df_codes_dev_ner.shape

(3947, 6)

Now, using the character start-end positions of each sentence from the CodiEsp corpus (see `datasets/CodiEsp-Sentence-Split.ipynb`), we annotate the sentences with CIE-Procedimiento codes. Also, using BETO tokenizer, each sentence is converted into a sequence of subwords, which are further converted into vocabulary indices (input IDs) and segments arrays (BERT input tensors). We also generate a *fragments* dataset indicating the number of produced annotated sentences for each document.

In [56]:
# Sentence-Split information
ss_corpus_path = "../datasets/CodiEsp-SSplit-text/"

### Training corpus

In [57]:
label_list = list(df_codes_train_dev_abs["code"])

In [58]:
len(label_list)

29332

In [59]:
len(set(label_list))

2194

In [60]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb_encoder = MultiLabelBinarizer()
mlb_encoder.fit([label_list])

MultiLabelBinarizer(classes=None, sparse_output=False)

In [61]:
# Number of distinct codes
num_labels = len(mlb_encoder.classes_)

In [62]:
num_labels

2194

Only training texts that are annotated with CIE-Diagnóstico codes are considered:

In [63]:
len(set(df_text_train["doc_id"]) - set(df_codes_train_ner["doc_id"]))

0

In [64]:
train_doc_list = sorted(set(df_codes_train_ner["doc_id"]))

In [65]:
len(train_doc_list)

500

In [66]:
# Sentence-Split data

In [67]:
%%time
ss_sub_corpus_path = ss_corpus_path + "train/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_train = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 13.4 ms, sys: 4.12 ms, total: 17.5 ms
Wall time: 17.2 ms


In [68]:
%%time
train_ind, train_seg, train_y, train_frag, train_start_end_frag = ss_create_frag_input_data_bert(df_text=df_text_train, 
                                                  text_col=text_col, 
                                                  df_ann=df_codes_train_ner, doc_list=train_doc_list, ss_dict=ss_dict_train,
                                                  tokenizer=tokenizer, lab_encoder=mlb_encoder, seq_len=SEQ_LEN,
                                                  greedy=False)

100%|██████████| 500/500 [00:15<00:00, 18.70it/s]

CPU times: user 15.6 s, sys: 36.5 ms, total: 15.6 s
Wall time: 15.5 s


In [69]:
# Sanity check

In [70]:
train_ind.shape

(7731, 128)

In [71]:
train_seg.shape

(7731, 128)

In [72]:
train_y.shape

(7731, 2194)

In [73]:
len(train_frag)

500

In [74]:
len(train_start_end_frag)

7731

In [75]:
# Check n_frag distribution across texts
pd.Series(train_frag).describe()

count    500.000000
mean      15.462000
std        7.670608
min        2.000000
25%       10.000000
50%       14.000000
75%       19.000000
max       54.000000
dtype: float64

In [76]:
# Inspect a randomly selected text and its encoded version
check_id = np.random.randint(low=0, high=len(train_doc_list), size=1)[0]

In [77]:
check_id

444

In [78]:
train_doc_list[check_id]

'S1139-76322013000200009-1'

In [79]:
df_text_train[df_text_train["doc_id"] == train_doc_list[check_id]][text_col].values[0]

'Un caso habitual de mareo, con diagnóstico inhabitual\nNiña de 11 años sin antecedentes familiares ni personales de interés, que acude de urgencia al centro de salud refiriendo presentar, desde la noche anterior, sensación de movimiento de objetos a su alrededor, con temor y ansiedad. La exploración física es normal; otoscopia sin alteraciones. Tensión arterial y glucemia normales. Llama la atención la ausencia de componente vegetativo, inestabilidad, palidez ni otra sintomatología como acúfenos o tinnitus. No se encuentra focalidad neurológica, alteraciones del equilibrio ni de la marcha, ataxia ni otros signos cerebelosos. La paciente tenía el antecedente de una toma de Romilar® (dextrometorfano) la noche anterior y una historia familiar positiva. Con estos datos y una exploración física normal, se la envió en observación a su casa, citándola a la mañana siguiente en la consulta.\n\n'

In [80]:
check_id_frag = sum(train_frag[:check_id])
for i in range(check_id_frag, check_id_frag + train_frag[check_id]):
    print(mlb_encoder.inverse_transform(np.array([train_y[i]])), "\n")

[('f41.9', 'r42')] 

[()] 

[()] 

[('h93.19', 'r23.1')] 

[('r27.0',)] 

[()] 

[()] 



In [81]:
for i in range(check_id_frag, check_id_frag + train_frag[check_id]):
    print(list(zip([tokenizer._token_dict_inv[ind] for ind in train_ind[i]][1:len(train_start_end_frag[i])+1], 
               train_start_end_frag[i])))
    print("\n")

[('Un', (0, 2)), ('caso', (3, 7)), ('habitual', (8, 16)), ('de', (17, 19)), ('mar', (20, 23)), ('##eo', (23, 25)), (',', (25, 26)), ('con', (27, 30)), ('diagnóstico', (31, 42)), ('in', (43, 45)), ('##hab', (45, 48)), ('##it', (48, 50)), ('##ual', (50, 53)), ('Ni', (54, 56)), ('##ña', (56, 58)), ('de', (59, 61)), ('11', (62, 64)), ('años', (65, 69)), ('sin', (70, 73)), ('antecedentes', (74, 86)), ('familiares', (87, 97)), ('ni', (98, 100)), ('personales', (101, 111)), ('de', (112, 114)), ('interés', (115, 122)), (',', (122, 123)), ('que', (124, 127)), ('acu', (128, 131)), ('##de', (131, 133)), ('de', (134, 136)), ('urgencia', (137, 145)), ('al', (146, 148)), ('centro', (149, 155)), ('de', (156, 158)), ('salud', (159, 164)), ('refi', (165, 169)), ('##riendo', (169, 175)), ('presentar', (176, 185)), (',', (185, 186)), ('desde', (187, 192)), ('la', (193, 195)), ('noche', (196, 201)), ('anterior', (202, 210)), (',', (210, 211)), ('sensación', (212, 221)), ('de', (222, 224)), ('movimiento', 

In [82]:
check_id_frag = sum(train_frag[:check_id])
for frag in train_ind[check_id_frag:check_id_frag + train_frag[check_id]]:
    print(' '.join([tokenizer._token_dict_inv[ind] for ind in frag]), "\n")

[CLS] Un caso habitual de mar ##eo , con diagnóstico in ##hab ##it ##ual Ni ##ña de 11 años sin antecedentes familiares ni personales de interés , que acu ##de de urgencia al centro de salud refi ##riendo presentar , desde la noche anterior , sensación de movimiento de objetos a su alrededor , con temor y ansiedad . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

[CLS] La exploración física es normal [UNK] o ##tos ##co ##pia sin alteraciones . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [83]:
# Fragment labels distribution
pd.Series(np.sum(train_y, axis=1)).describe()

count    7731.000000
mean        0.931186
std         1.344309
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max        14.000000
dtype: float64

### Development corpus

Only development texts that are annotated with CIE-Diagnóstico codes are considered:

In [84]:
len(set(df_text_dev["doc_id"]) - set(df_codes_dev_ner["doc_id"]))

0

In [85]:
dev_doc_list = sorted(set(df_codes_dev_ner["doc_id"]))

In [86]:
len(dev_doc_list)

250

In [87]:
%%time
ss_sub_corpus_path = ss_corpus_path + "dev/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_dev = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 5.83 ms, sys: 3.97 ms, total: 9.79 ms
Wall time: 9.5 ms


In [88]:
%%time
dev_ind, dev_seg, dev_y, dev_frag, dev_start_end_frag = ss_create_frag_input_data_bert(df_text=df_text_dev, 
                                                  text_col=text_col, 
                                                  df_ann=df_codes_dev_ner, doc_list=dev_doc_list, ss_dict=ss_dict_dev,
                                                  tokenizer=tokenizer, lab_encoder=mlb_encoder, seq_len=SEQ_LEN,
                                                  greedy=False)

100%|██████████| 250/250 [00:07<00:00, 32.58it/s]


CPU times: user 7.74 s, sys: 28.1 ms, total: 7.76 s
Wall time: 7.72 s


In [89]:
# Sanity check

In [90]:
dev_ind.shape

(4107, 128)

In [91]:
dev_seg.shape

(4107, 128)

In [92]:
dev_y.shape

(4107, 2194)

In [93]:
len(dev_frag)

250

In [94]:
len(dev_start_end_frag)

4107

In [95]:
# Check n_frag distribution across texts
pd.Series(dev_frag).describe()

count    250.000000
mean      16.428000
std        8.409023
min        4.000000
25%       11.000000
50%       15.000000
75%       20.750000
max       65.000000
dtype: float64

In [96]:
# Inspect a randomly selected text and its encoded version
check_id = np.random.randint(low=0, high=len(dev_doc_list), size=1)[0]

In [97]:
check_id

232

In [98]:
dev_doc_list[check_id]

'S1698-44472004000200012-1'

In [99]:
df_text_dev[df_text_dev["doc_id"] == dev_doc_list[check_id]][text_col].values[0]

'Paciente mujer de 67 años de edad que fue remitida por su médico de cabecera a nuestra consulta por presentar una tumoración oral de 20 años de evolución. La lesión era asintomática y había crecido en el último año, produciéndole una deformidad estética facial e interfiriéndole la masticación. Como antecedente personal cabía destacar histerectomía más doble anisectomía hacía 10 años. No refería hábito tabáquico, ni consumo de alcohol.\nEn la exploración física se constataba asimetría facial. En el examen intraoral se apreciaba una tumoración pediculada en cresta gingival mandibular izquierda correspondiente al espacio edéntulo de 37, de 6-8 cm de diámetro, bien delimitada, parcialmente cubierta por una mucosa de coloración blanquecina y de consistencia duro-elástica. Además presentaba movilidad por desplazamiento de 35, 36 y 38 y restos radiculares de 25, 26 y 27. La exploración cervical era negativa.\nEl examen radiológico con ortopantomografía ponía de manifiesto el desplazamiento d

In [100]:
check_id_frag = sum(dev_frag[:check_id])
for i in range(check_id_frag, check_id_frag + dev_frag[check_id]):
    print(mlb_encoder.inverse_transform(np.array([dev_y[i]])), "\n")

[()] 

[('m95.2',)] 

[('z90.710',)] 

[('f10.10',)] 

[('q67.0',)] 

[()] 

[()] 

[()] 

[()] 

[('d16.9',)] 

[()] 



In [101]:
for i in range(check_id_frag, check_id_frag + dev_frag[check_id]):
    print(list(zip([tokenizer._token_dict_inv[ind] for ind in dev_ind[i]][1:len(dev_start_end_frag[i])+1], 
               dev_start_end_frag[i])))
    print("\n")

[('Paci', (0, 4)), ('##ente', (4, 8)), ('mujer', (9, 14)), ('de', (15, 17)), ('6', (18, 19)), ('[UNK]', (19, 20)), ('años', (21, 25)), ('de', (26, 28)), ('edad', (29, 33)), ('que', (34, 37)), ('fue', (38, 41)), ('remi', (42, 46)), ('##tida', (46, 50)), ('por', (51, 54)), ('su', (55, 57)), ('médico', (58, 64)), ('de', (65, 67)), ('cabecera', (68, 76)), ('a', (77, 78)), ('nuestra', (79, 86)), ('consulta', (87, 95)), ('por', (96, 99)), ('presentar', (100, 109)), ('una', (110, 113)), ('tumor', (114, 119)), ('##ación', (119, 124)), ('oral', (125, 129)), ('de', (130, 132)), ('20', (133, 135)), ('años', (136, 140)), ('de', (141, 143)), ('evolución', (144, 153)), ('.', (153, 154))]


[('La', (155, 157)), ('lesión', (158, 164)), ('era', (165, 168)), ('asi', (169, 172)), ('##nto', (172, 175)), ('##mática', (175, 181)), ('y', (182, 183)), ('había', (184, 189)), ('crecido', (190, 197)), ('en', (198, 200)), ('el', (201, 203)), ('último', (204, 210)), ('año', (211, 214)), (',', (214, 215)), ('produc

In [102]:
check_id_frag = sum(dev_frag[:check_id])
for frag in dev_ind[check_id_frag:check_id_frag + dev_frag[check_id]]:
    print(' '.join([tokenizer._token_dict_inv[ind] for ind in frag]), "\n")

[CLS] Paci ##ente mujer de 6 [UNK] años de edad que fue remi ##tida por su médico de cabecera a nuestra consulta por presentar una tumor ##ación oral de 20 años de evolución . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

[CLS] La lesión era asi ##nto ##mática y había crecido en el último año , produci ##éndole una deform ##idad estética facial e inter ##fi ##ri ##éndole la mas ##tica ##ción . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [103]:
# Fragment labels distribution
pd.Series(np.sum(dev_y, axis=1)).describe()

count    4107.000000
mean        0.833455
std         1.253935
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max        15.000000
dtype: float64

### Train-Dev abstracts corpus

In [104]:
len(set(df_text_all_abs["doc_id"]) - set(df_codes_d_all_abs["doc_id"]))

0

In [105]:
all_abs_doc_list = sorted(set(df_codes_d_all_abs["doc_id"]))

In [106]:
len(all_abs_doc_list)

15258

In [107]:
%%time
all_abs_ind, all_abs_seg, all_abs_y, all_abs_frag = create_frag_input_data_bert(df_text=df_text_all_abs, text_col=text_col, 
                                     df_label=df_codes_d_all_abs, doc_list=all_abs_doc_list, tokenizer=tokenizer, 
                                     lab_encoder=mlb_encoder, seq_len=SEQ_LEN)

100%|██████████| 15258/15258 [00:38<00:00, 395.14it/s]


CPU times: user 39.1 s, sys: 183 ms, total: 39.3 s
Wall time: 38.8 s


In [108]:
all_abs_ind.shape

(15258, 128)

In [109]:
all_abs_seg.shape

(15258, 128)

In [110]:
all_abs_y.shape

(15258, 2194)

In [111]:
len(all_abs_frag)

15258

In [112]:
# Check n_frag distribution across texts
pd.Series(all_abs_frag).describe()

count    15258.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
dtype: float64

In [113]:
# Inspect a randomly selected text and its encoded version
check_id = np.random.randint(low=0, high=len(all_abs_doc_list), size=1)[0]

In [114]:
check_id

8982

In [115]:
all_abs_doc_list[check_id]

'lil-34552'

In [116]:
df_text_all_abs[df_text_all_abs["doc_id"] == all_abs_doc_list[check_id]][text_col].values[0]

'Se presentan dos casos de siringomas eruptivos. El primero asociado a una afección psiquiátrica, con sintomatología pruriginosa y ubicación poco común. El segundo por presentar las dos variantes clínicas en el mismo paciente. Se realiza una revisión de las características histoquímicas y ultraestructurales que establecen la génesis glandular ecrina y se hace hincapié en la necesidad de comunicar asociacines de patologías con causal genético en estudio'

In [117]:
check_id_frag = sum(all_abs_frag[:check_id])
for i in range(check_id_frag, check_id_frag + all_abs_frag[check_id]):
    print(mlb_encoder.inverse_transform(np.array([all_abs_y[i]])), "\n")

[('f99', 'q90.9')] 



In [118]:
check_id_frag = sum(all_abs_frag[:check_id])
for frag in all_abs_ind[check_id_frag:check_id_frag + all_abs_frag[check_id]]:
    print(' '.join([tokenizer._token_dict_inv[ind] for ind in frag]), "\n")

[CLS] Se presentan dos casos de sir ##ingo ##mas eru ##pti ##vos . El primero asociado a una afección psiquiá ##trica , con sin ##toma ##tología pru ##ri ##gin ##osa y ubicación poco común . El segundo por presentar las dos variantes clínicas en el mismo paciente . Se realiza una revisión de las características hist ##o ##química ##s y ultra ##estructura ##les que establecen la gén ##esis gl ##and ##ular ec ##rina y se hace hincapié en la necesidad de comunicar asocia ##cine ##s de pato ##logías con causa ##l genético en estudio [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 



In [119]:
# Fragment labels distribution
pd.Series(np.sum(all_abs_y, axis=1)).describe()

count    15258.000000
mean         1.377376
std          0.746101
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max         10.000000
dtype: float64

### Training & Development & Abstracts corpus

We merge the previously generated datasets:

In [120]:
# Indices
train_dev_abs_ind = np.concatenate((train_ind, dev_ind, all_abs_ind))

In [121]:
train_dev_abs_ind.shape

(27096, 128)

In [122]:
# Segments
train_dev_abs_seg = np.concatenate((train_seg, dev_seg, all_abs_seg))

In [123]:
train_dev_abs_seg.shape

(27096, 128)

In [124]:
# y
train_dev_abs_y = np.concatenate((train_y, dev_y, all_abs_y))

In [125]:
train_dev_abs_y.shape

(27096, 2194)

## Fine-tuning

Using the corpus of labeled sentences, we fine-tune the model on a multi-label sentence classification task.

In [123]:
from keras.backend.tensorflow_backend import set_session

# Prevent GPU memory allocation problems
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [124]:
from keras_bert import load_trained_model_from_checkpoint

model = load_trained_model_from_checkpoint(
    config_file=config_path, 
    checkpoint_file=checkpoint_path, 
    training=training,                                       
    trainable=trainable, 
    seq_len=SEQ_LEN
)

In [125]:
model.inputs

[<tf.Tensor 'Input-Token:0' shape=(?, 128) dtype=float32>,
 <tf.Tensor 'Input-Segment:0' shape=(?, 128) dtype=float32>]

In [126]:
model.outputs

[<tf.Tensor 'Encoder-12-FeedForward-Norm/add_1:0' shape=(?, 128, 768) dtype=float32>]

In [127]:
from keras.layers import Dense, Activation
from keras.models import Model
from keras.initializers import glorot_uniform
from keras_bert.layers import Extract

dense_cls = Extract(index=0, name='Extract')(model.output) # In order to extract CLS token embedding
dense_out = Dense(units=num_labels, kernel_initializer=glorot_uniform(seed=random_seed))(dense_cls) # Multi-label classification
outputs = Activation('sigmoid')(dense_out)

model = Model(model.inputs, outputs)

In [128]:
model.inputs

[<tf.Tensor 'Input-Token:0' shape=(?, 128) dtype=float32>,
 <tf.Tensor 'Input-Segment:0' shape=(?, 128) dtype=float32>]

In [129]:
model.outputs

[<tf.Tensor 'activation_1/Sigmoid:0' shape=(?, 2194) dtype=float32>]

In [132]:
# Sample weights

In [130]:
n_train_dev_frags = sum(train_frag) + sum(dev_frag)

In [131]:
n_train_dev_frags

11838

In [132]:
train_dev_abs_weights = np.array([train_weight] * n_train_dev_frags + [all_abs_weight] * (train_dev_abs_y.shape[0] - 
                                                                                      n_train_dev_frags))

In [ ]:
%%time
from keras_radam import RAdam

model.compile(
    optimizer=RAdam(learning_rate=LR),
    loss='binary_crossentropy'
)

history = model.fit(
    x=[train_dev_abs_ind, train_dev_abs_seg],
    y=train_dev_abs_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    sample_weight=train_dev_abs_weights,
    shuffle=True
)

Epoch 1/24
27096/27096 [==============================] - 470s 17ms/step - loss: 0.2808
Epoch 2/24
27096/27096 [==============================] - 463s 17ms/step - loss: 0.0134
Epoch 3/24
27096/27096 [==============================] - 463s 17ms/step - loss: 0.0090
Epoch 4/24
27096/27096 [==============================] - 463s 17ms/step - loss: 0.0083
Epoch 5/24
27096/27096 [==============================] - 463s 17ms/step - loss: 0.0072
Epoch 6/24
27096/27096 [==============================] - 463s 17ms/step - loss: 0.0060
Epoch 7/24
27096/27096 [==============================] - 463s 17ms/step - loss: 0.0049
Epoch 8/24
27096/27096 [==============================] - 463s 17ms/step - loss: 0.0040
Epoch 9/24
27096/27096 [==============================] - 463s 17ms/step - loss: 0.0033
Epoch 10/24
27096/27096 [==============================] - 463s 17ms/step - loss: 0.0027
Epoch 11/24
27096/27096 [==============================] - 463s 17ms/step - loss: 0.0022
Epoch 12/24
27096/27096 [=====

## Test set predictions

Finally, the predictions made by the model on the test set are saved. For this purpose, firstly, each sentence from the test corpus must be converted into a sequence of subwords (input IDs and attention mask arrays). Then, the predictions made by the model at the sentence-level are saved, to be further evaluated at document-level (see `results/CodiEsp-D/Evaluation.ipynb`).

In [126]:
%%time
test_path = corpus_path + "test/text_files/"
test_files = [f for f in os.listdir(test_path) if os.path.isfile(test_path + f)]
test_data = load_text_files(test_files, test_path)
df_text_test = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in test_files], 'raw_text': test_data})

CPU times: user 43.1 ms, sys: 0 ns, total: 43.1 ms
Wall time: 7.15 ms


In [127]:
df_text_test.shape

(250, 2)

In [128]:
df_text_test.head()

,doc_id,raw_text
0,S0365-66912007000900014-1,Paciente varón de 34 años de edad diagnosticad...
1,S0211-69952014000200012-1,"Un varón de 48 años, de raza caucásica, con IR..."
2,S1139-76322017000200009-1,Presentamos el caso clínico de un niño de cinc...
3,S0210-48062010000100019-1,"Paciente varón de 53 años, diagnosticado de es..."
4,S1130-14732005000500006-1,Se trata de un varón de 20 años diagnosticado ...


In [129]:
df_text_test.raw_text[0]

'Paciente varón de 34 años de edad diagnosticado de varicela tres semanas antes ya resuelta sin complicaciones. Acude a urgencias por presentar disminución de agudeza visual en su ojo izquierdo.\nEn la exploración oftalmológica presenta una agudeza visual corregida de 1 en el ojo derecho (OD) y de 0,6 en el ojo izquierdo (OI). El estudio con lámpara de hendidura demuestra en el OI un tyndall celular de 4+, precipitados queráticos inferiores (3+) y sin presentar la cornea tinción con fluoresceína, siendo normal el OD. La presión intraocular fue de 16mmHg en ambos ojos.\nEn la exploración fundoscópica inicial del OI se aprecia leve vitritis (1+) sin focos de retinitis.\nSe instaura tratamiento tópico con corticoides y midriáticos. A los 2 días se observa leve disminución del tyndall celular (3+) en cámara anterior pero en fondo de ojo aparece un foco periférico de retinitis necrotizante en el área temporal asociado a vasculitis retiniana.\nSe ingresa al paciente y se instaura tratamiento

In [130]:
test_doc_list = sorted(set(df_text_test["doc_id"]))

In [131]:
len(test_doc_list)

250

In [132]:
%%time
ss_sub_corpus_path = ss_corpus_path + "test/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_test = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 69.5 ms, sys: 0 ns, total: 69.5 ms
Wall time: 11.5 ms


In [133]:
%%time
test_ind, test_seg, _, test_frag, _ = ss_create_frag_input_data_bert(df_text=df_text_test, 
                                                  text_col=text_col,
                                                  # Since labels are ignored, we pass df_codes_train_ner as df_ann
                                                  df_ann=df_codes_train_ner, doc_list=test_doc_list, ss_dict=ss_dict_test,
                                                  tokenizer=tokenizer, lab_encoder=mlb_encoder, seq_len=SEQ_LEN,
                                                  greedy=False)

100%|██████████| 250/250 [00:01<00:00, 180.60it/s]

CPU times: user 2.15 s, sys: 12.2 ms, total: 2.17 s
Wall time: 1.43 s


In [113]:
%%time
test_preds = model.predict([test_ind, test_seg])

CPU times: user 14.9 s, sys: 1.68 s, total: 16.6 s
Wall time: 16.9 s


In [114]:
test_preds.shape

(3950, 727)

In [134]:
results_dir_path = "../results/CodiEsp-D/"

In [178]:
%%time
np.save(file=results_dir_path + "predictions/beto_r_seed_" + str(random_seed) + "_test_preds.npy", arr=test_preds)

CPU times: user 2.21 ms, sys: 4.65 ms, total: 6.87 ms
Wall time: 6.02 ms


In [135]:
# To be further used when evaluating model performance at document level
np.save(file=results_dir_path + "beto_test_frags.npy", arr=test_frag)